![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)



# Collecting data with a Phidgets Plant Kit 

This is a utility notebook, to collect data from sensors around a plant and store in an online spreadsheet. The spreadsheet can then be accessed by anyone with the web address, to analysize the data.

Ask M. Lamoureux for details. May 2023.

<h2 style="color: red">IMPORTANT</h2>
You MUST click the <span style="color:red">BIG, RED DISCONNECT BUTTON</span> at the end of this notebook when you are done. This tells the software to release the Phidget device, so it can be used by other notebooks you might try later.

***
## Stepping up your plant kit 

It takes quite a bit of setup to use this notebook appropriately.

- you need the Phidget VINT device, with three sensors attached
    - the temperature/humidity sensor (device type HUM1001_01)
    - the light sensor (device type LUX1000_0)
    - the moisture sensor (device type HUM1100)
- a computer (Mac, Windows, Raspberry Pi) with USB connection and internet connection
- the Chrome or Chromium Web browser
- ideally, Jupyter software installed on the computer, to run independently of the Callysto Hub

Phidgets has step by step instructions here: https://www.phidgets.com/education/learn/projects/plant-kit/assemble/

This what a typical setup looks like:

<img src="images/plant2.jpg" alt="A plant with sensor" width="400"/>
<div align="center">

 
A basement window with our plant.
</div>


The notebook runs both Python code and Javascript (JS) code. The JS code is the front end, that talks to the browser and the Phidget devices collecting data. The Python code is the back end, taking care of things like dates, data format, and saving the data onto the online web spreadsheet.

One very tricky aspect is getting the JS and Python engines talking to each other. This makes use of the "Comms" package in Jupyter that manages the communication between these two engines. One key feature of the Python engine (or kernel) is that when it is executing code, it cannot be interrupted by the JS engine. So even if you have a simple loop in Python, or a sleep command to pause Python, the JS engine cannot interrupt it to give it new data.

So, our trick is to set a timer in JS that gathers data every few minutes, then sends a message to Python saying "deal with this data." This triggers a Python code that looks at the data and decides if it is time to save it to the web. So in fact, the Python is not "doing" anything most of the time, so JS is not actually interrupting it. Instead, it is telling Python "now is the time to deal with this data" and the Python engine kicks in. 

***
### Setting up your Spreadsheet

The spreadsheet can be viewed here:

https://ethercalc.net/MyWeirdName01

The 4 numbers following the date and time are temperature, humidity, moisture in the soil, and the light level. Here is a sample.

|Date	|Time	|Temperature	|Humidity	|Moisture	|Luminance |
| ------ | ------ | ------ | ------ | ------ | ------ | 
|2023-04-30 |21:43:42	|20.64	|50.29	|0.199	|0.0193 |
|2023-04-30	|21:43:43	|20.64	|50.29	|0.199	|0.0193 |
|2023-04-30	|21:43:47	|20.66	|50.26	|0.198	|0.044 |
|2023-04-30	|21:43:47	|20.64	|50.26	|0.198	|0.044 |
|2023-04-30	|21:43:52	|20.66	|50.24	|0.199	|0.043 |

***
### Here are the steps to get your Phidget Plants up and running in Jupyter Notebook 

You must be running on Chrome or Chromium on your laptop or desktop, and the Phidget VINT device needs to be connected to the USB port on your computer. 

The VINT device must have three sensors attached: the temperature/humidity sensor, the light sensor, and the soil moisture sensor.

The main steps are:

0. Import some Python libraries
1. Import the Phidget library, in Javascript
2. Define and open the USB connection
3. Open the Phidget devices
4. Set up Python code to post data to the online web spreadsheet
5. Set up the Comms channel, so JS and Python can talk to each other
6. Initiate the JS engine to start collecting data
7. Let it run for a few hours, or days, to collect data
8. When we are done, close the devices and USB connection


#### Step 0. Python Libraries

Let's import a few Python libraries that we need. Click on `▶Run`

In [1]:
import ipywidgets as widgets
from time import sleep
import requests
from datetime import datetime, timedelta

#### Step 1. Phidgets library.

Now import the Phidgets library. 

We are using a special library for debugging purposes. We will need to update this when the Phidgets company finishes their testing. 

Special library is here: https://www.phidgets.com/downloads/testing/phidget22

In [2]:
%%js
requirejs.config({
    paths: { 
        'phidget22': ['https://www.phidgets.com/downloads/testing/phidget22'], 
    },                                         
});
require(['phidget22'], (phidget22) => {
   window.phidget22 = phidget22; 
});

<IPython.core.display.Javascript object>

In [3]:
## We pause for a second here, to allow the library to load
sleep(1)

#### Step 2. Open USB

We now open a USB connection.  A pop up window will appear and click on the the device name to open the USB connection, then click "Open"

In [4]:
%%js

if (window.usbconn === undefined) {
    
    element.text("Creating a new USB Connection.");
    
    window.usbconn = new phidget22.USBConnection();
    
    usbconn.connect().then(() => {
        usbconn.requestWebUSBDeviceAccess();
    }).catch(err => {
        window.usbconn.delete();
        element.append("Error connecting to USB" + err);
    });
}

<IPython.core.display.Javascript object>

In [5]:
## We rest for a bit while the USB connects
sleep(1)

### Confirm the connection 

In [6]:
%%js
element.text(usbconn.connected);

<IPython.core.display.Javascript object>

#### Step 3. Connect the sensors

We make a request to open the four different sensors. We pause between each one, and check that they are connected. 

In [7]:
%%js
window.humSensor = new phidget22.HumiditySensor();
humSensor.open();

<IPython.core.display.Javascript object>

In [8]:
sleep(1)

In [9]:
%%js
element.text("Is the humidity sensor attached? " + humSensor.attached);

<IPython.core.display.Javascript object>

In [10]:
%%js
window.tempSensor = new phidget22.TemperatureSensor();
tempSensor.open();

<IPython.core.display.Javascript object>

In [11]:
sleep(1)

In [12]:
%%js
element.text("Is the temperature sensor attached? " + tempSensor.attached);

<IPython.core.display.Javascript object>

In [13]:
%%js
window.vrSensor = new phidget22.VoltageRatioInput();
vrSensor.open();

<IPython.core.display.Javascript object>

In [14]:
sleep(1)

In [15]:
%%js
element.text("Is the moisture sensor attached? " + vrSensor.attached);

<IPython.core.display.Javascript object>

In [16]:
%%js
window.liteSensor = new phidget22.LightSensor();
liteSensor.open();

<IPython.core.display.Javascript object>

In [17]:
sleep(1)

In [18]:
%%js
element.text("Is the light sensor attached? " + liteSensor.attached);

<IPython.core.display.Javascript object>

#### Step 4: Python code to post online

We set up some code to post data to the online spreadsheet.

The data comes from the Comms channel in the form of a dictionary:

comm_data = {'time': date-time value, 
             'temp': value, 
             'humidity': value,
             'ratio': value,
             'lux': value}

This will get sent to the function **post_stuff()**, which decides when to post it based on how much time has elapsed. We record data more frequently in the daytime, since the sunlight makes interesting things happen. 

We also have a widget to keep track of whether data has been posted. 

In [39]:
# the URL of the spreadsheet, for posting
post_url = 'https://ethercalc.net/_/MyWeirdName01'

# set a reminder of when the last data was stored
last_time = datetime.now() - timedelta(hours=2)

# check if it is daytime
def is_daylight(): 
    if (datetime.now().hour > 8) and (datetime.now().hour < 20):
        return True
    return False

# a widget to monitor the data being posted
monitor = widgets.Text(
    value="No recent post yet",
    placeholder="blank",
    description="Latest data:",
    disabled = False
    )

# here we grab the data results from comm, check if it is time to post, then post. 
def post_stuff(result):
    global last_time
    if is_daylight():
        wait_time = 14*60 # in daytime, we post every 15 minute (wait 14). 
    else:
        wait_time = 59*60 # in nighttime, post every 60 minutes (wait 59)
    if (datetime.now()-last_time).seconds < wait_time:
        return
    last_time = datetime.now()
    data = datetime.now().strftime("%Y-%m-%d,%H:%M:%S")
    data += ',' + str(result['temp'])
    data += ',' + str(result['humidity'])
    data += ',' + str(result['ratio'])
    data += ',' + str(result['lux'])
    monitor.value = data    
    r = requests.post(post_url, data= data) ## we could return a status code here...

#### Step 5: Open a comms channel 

This connects the Javascript and Python engines.

Javascript (front end) and Python (back end) run separately on the Jupyter system. We open a Comms channel between the two so JS can pass the sensor data to Python, then Python posts it onto the web.  

There is an issue that Python runs as a single thread in Jupyter. So if you put a "sleep" command in a Python loop, no other code can access the Jupyter kernel during that loop. The sensor data is then not recorded. 

The solution is to create a timer loop in Javascript, so that every 5 minutes it sends a message to the Python kernel. Basically, the message says "hey Python, here is the latest data. Please save it to the cloud." Then the Python code can run and save the data. 

This way there is no loop or sleep in Python, so nothing to interupt. The loop is in JS (See "setIntevalTimer in the JS code below).


##### First open the front end of the Comms channel, in JavaScript

In [20]:
%%js
Jupyter.notebook.kernel.comm_manager.register_target('my_comm_target',
    function(comm, msg) {
        // comm is the frontend comm instance, msg is the comm_open message, which can carry data
        // We don't do anything with the above opening msg
        // Next, we register a handler to deal with later messages.
        // Basically, on receiving a message, we start a timer that posts a send command,
        // which sends the data in a message to the Python back end
        comm.on_msg(function(msg) {
            console.log("Comm message received " + msg.content.data.foo);
            // this function is called by the timer, sends data to the back end
            function myTimer() {      
                const d = new Date();
                comm.send({'time': d.toLocaleString(), 
                           'temp':tempSensor.temperature, 
                           'humidity':humSensor.humidity,
                           'ratio':vrSensor.sensorValue,
                           'lux':liteSensor.illuminance});
            } 
            let myVar = setInterval(myTimer, 300000); // run every 5 minutes (300sec)
        });
        comm.on_close(function(msg) {return 0;});
    });

<IPython.core.display.Javascript object>

##### Next, open the back end of the Comms channel, in Python

In [21]:
from ipykernel.comm import Comm

# a global variable we use to keep the data from the front end comm channel
comm_data = "Not set yet"  

# Connect to the comm channel in the front end, to the Python
my_comm = Comm(target_name='my_comm_target', data={'foo': 1})

# Add a callback for received messages. This will call post_stuff to store the data
@my_comm.on_msg
def _recv(msg):
    global comm_data
    comm_data = msg['content']['data']  # this is the data in the comm message
    post_stuff(comm_data)

#### Step 6. Start the JS engine

We send a message to the Comms channel, and this will start the data collection and posting. 

We can watch the monitor widget to see the most recent data being posted. 

In [22]:
monitor

Text(value='No recent post yet', description='Latest data:', placeholder='blank')

In [23]:
## This command starts the JS loop running.
my_comm.send({'foo':55})

W can check the status of the Phidgets. Run the following code to check the humidity sensor.

In [ ]:
%%js
element.text(humSensor.humidity);

#### Step 7. Wait

The data should be collecting now. You can watch the monitor above, it should update every 10 or 15 minutes in the daytime, and every hour at night. 

You can also check the online spreadsheet, to see that data is getting entered. 
https://ethercalc.net/MyWeirdName23



#### Step 8. Closing down the sensors

It is **really important** to close the sensor now, as otherwise they will keep busy forever, always trying to update the sliders with the latest values. So, don't skip this next step.

The following cell creates a button that you can click to close the Phidgets. Click it once you are all done with the Phidgets in this notebook. 

In [17]:
def doDisconnect(b):
    display(Javascript("""
        (async () => {
            await humSensor.close();
            await tempSensor.close();
            await vrSensor.close();
            await liteSensor.close();
            usbconn.close();
            usbconn.delete();
            delete window.usbconn;
            element.text("You have disconnected the Phidgets.");
        })();
    """))

run_button = widgets.Button(
    description = 'IMPORTANT: Click to disconnect', 
        button_style='danger',layout=widgets.Layout(width='50%', height='80px')
)
print("Press this button when you are done, to disconnect the Phidgets")
run_button.on_click(doDisconnect)

display(run_button)

Press this button when you are done, to disconnect the Phidgets


Button(button_style='danger', description='IMPORTANT: Click to disconnect', layout=Layout(height='80px', width…


##### Confirm

You can confirm the phidget is open or closed by running the following cell. 

If it is still attached (true), try clicking the button above, again.

In [ ]:
%%js
element.text("Is the humidity sensor attached? " + humSensor.attached);
element.append("<p>Is the temperature sensor attached? " + tempSensor.attached + "</p>");
element.append("Is the moisture sensor attached? " + vrSensor.attached +"</p>");
element.append("Is the light sensor attached? " + liteSensor.attached);

## Conclusion

We have shown how to run a utility to grab Phidget sensor data and post it on the web. Let us begin to chart out the data we have collected. NOTEBOOK??

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)